In [1]:
%load_ext autoreload
%autoreload 2

In [19]:
import cvxpy as cp
import numpy as np
import torch
import torch.nn as nn

from multiprocessing import Pool

from pideq.net import PIDEQ
from pideq.utils import load_from_wandb

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [4]:
deq = load_from_wandb(PIDEQ(np.pi / 2, n_states=200), '2qka16vc', project='pideq-nls', model_fname='model_best')
deq

PIDEQ(
  (A): Linear(in_features=200, out_features=200, bias=True)
  (B): Linear(in_features=2, out_features=200, bias=True)
  (C): Linear(in_features=200, out_features=2, bias=True)
  (D): Linear(in_features=2, out_features=2, bias=True)
)

In [5]:
A0 = deq.A.weight.detach().numpy()
np.linalg.norm(A0, ord=np.inf)

7.147842

### Full optimization

In [6]:
%%time
kappa = .9
A = cp.Variable(A0.shape, complex=False)

constraint = [cp.norm(A, p='inf') <= kappa,]

prob = cp.Problem(cp.Minimize(cp.norm(A - A0, p='fro')), constraint)
prob.solve(verbose=False)
prob.status

CPU times: user 3.07 s, sys: 27 ms, total: 3.09 s
Wall time: 3.09 s


'optimal'

In [7]:
np.linalg.norm(A.value - A0, ord='fro')

2.7698592994118885

### Decomposition in rows

In [8]:
%%time
kappa = .9
A_rows = list()

for i in range(A0.shape[0]):
    a = cp.Variable(A0.shape[1], complex=False)

    constraint = [cp.norm(a, p=1) <= kappa,]

    prob = cp.Problem(cp.Minimize(cp.norm(a - A0[i], p=2)), constraint)
    prob.solve(verbose=False)

    A_rows.append(a.value)
A = np.vstack(A_rows)

CPU times: user 1.87 s, sys: 19.7 ms, total: 1.89 s
Wall time: 1.29 s


In [9]:
np.linalg.norm(A, ord=np.inf)

0.8999999999421489

In [10]:
np.linalg.norm(A - A0, ord='fro')

2.7698593009135677

### Simultaneous

In [37]:
%%time
kappa = .9
A = cp.Variable(A0.shape, complex=False)

cost = None
constraints = list()
for i in range(A0.shape[0]):
    constraints.append(cp.norm(A[i], p=1) <= kappa)
    if cost is None:
        cost = cp.norm(A[i] - A0[i], p=2)
    else:
        cost += cp.norm(A[i] - A0[i], p=2)

prob = cp.Problem(cp.Minimize(cost), constraints)
prob.solve(verbose=False)

CPU times: user 2.2 s, sys: 17.9 ms, total: 2.22 s
Wall time: 2.22 s


37.181250557840045

In [12]:
np.linalg.norm(A.value, ord=np.inf)

0.899999999669481

In [13]:
np.linalg.norm(A.value - A0, ord='fro')

2.7698593000384606

### Parallel (multiprocessing.Pool)

In [91]:
%%time
kappa = .9
A_rows = list()

a = cp.Variable(A0.shape[1], complex=False)
a0 = cp.Parameter(A0.shape[1], complex=False)

constraint = [cp.norm(a, p=1) <= kappa,]

prob = cp.Problem(cp.Minimize(cp.norm(a - a0, p=2)), constraint)

pool = Pool()

def get_a(i):
    a0.value = A0[i]
    prob.solve()

    return a.value

A_rows = pool.map(get_a, list(range(A0.shape[0])))
A = np.vstack(A_rows)

CPU times: user 40.9 ms, sys: 547 ms, total: 588 ms
Wall time: 721 ms


In [92]:
np.linalg.norm(A, ord=np.inf)

0.8999999999421489

In [93]:
np.linalg.norm(A - A0, ord='fro')

2.7698593009135677